In [ ]:
# run this cell to download the right packages (only needed once)
!python --version

!pip install cifar10
!pip install imageio numpy scipy    
!pip install git+https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks

Python 3.9.16
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to /tmp/pip-req-build-tbs74fkr
  Running command git clone --filter=blob:none --quiet https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks /tmp/pip-req-build-tbs74fkr
  Resolved https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to commit 28caa7cde240e354fd7b87280450fd233cd494c3
  Preparing metadata (setup.py) ... done


In [ ]:
import time
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision import datasets, transforms

from core_qnn.quaternion_layers import QuaternionConv, QuaternionLinear
from core_qnn.quaternion_ops import check_input, q_normalize

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
%%time

# import and download the CIFAR10 dataset
transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5),(.5,.5,.5))])
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
CPU times: user 1.41 s, sys: 273 ms, total: 1.68 s
Wall time: 1.68 s


In [ ]:
class InvalidKernelShape(RuntimeError):
  """Base class to generate custom exception if generating kernel failed."""

  def __init__(self, error_message):
    """ Construct custom error with custom error message.
    :param error_message: The custom error message.
    """
    super().__init__(error_message)

class InvalidInput(RuntimeError):
  """Base class to generate custom exception if input is invalid."""

  def __init__(self, error_message):
    """ Construct custom error with custom error message.
    :param error_message: The custom error message.
    """
    super().__init__(error_message)

In [ ]:
class QuaternionConvolution(nn.Module):
  """Reproduction class of the quaternion convolution layer."""

  ALLOWED_DIMENSIONS = (2, 3)

  def __init__(self, in_channels, out_channels, kernel_size, stride, dimension=2, padding=0, dilation=1, groups=1, bias=True):
    """Create the quaterion convolution layer."""
    super(QuaternionConvolution, self).__init__()

    self.in_channels = np.floor_divide(in_channels, 4)
    self.out_channels = np.floor_divide(out_channels, 4)

    self.groups = groups
    self.stride = stride
    self.padding = padding
    self.dilation = dilation

    self.kernel_size = self.get_kernel_shape(kernel_size, dimension)
    self.weight_shape = self.get_weight_shape(self.in_channels, self.out_channels, self.kernel_size)

    self._weights = self.weight_tensors(self.weight_shape, kernel_size)
    self.r_weight, self.k_weight, self.i_weight, self.j_weight = self._weights
    
    if bias:
      self.bias = nn.Parameter(torch.Tensor(out_channels))
      nn.init.constant_(self.bias, 0)

  def forward(self, x):
    """Apply forward pass of input through quaternion convolution layer."""
    cat_kernels_4_r = torch.cat([self.r_weight, -self.i_weight, -self.j_weight, -self.k_weight], dim=1)
    cat_kernels_4_i = torch.cat([self.i_weight,  self.r_weight, -self.k_weight, self.j_weight], dim=1)
    cat_kernels_4_j = torch.cat([self.j_weight,  self.k_weight, self.r_weight, -self.i_weight], dim=1)
    cat_kernels_4_k = torch.cat([self.k_weight,  -self.j_weight, self.i_weight, self.r_weight], dim=1)

    cat_kernels_4_quaternion   = torch.cat([cat_kernels_4_r, cat_kernels_4_i, cat_kernels_4_j, cat_kernels_4_k], dim=0)

    if x.dim() == 3:
        convfunc = F.conv1d
    elif x.dim() == 4:
        convfunc = F.conv2d
    elif x.dim() == 5:
        convfunc = F.conv3d
    else:
        raise InvalidInput("Given input channels do not match allowed dimensions")

    return convfunc(x, cat_kernels_4_quaternion, self.bias, self.stride, self.padding, self.dilation, self.groups)

  @staticmethod
  def weight_tensors(weight_shape, kernel_size):
    """Create and initialise the weight tensors according to quaternion rules."""
    modulus = nn.Parameter(torch.Tensor(*weight_shape))
    modulus = nn.init.xavier_uniform_(modulus, gain=1.0)

    i_weight = 2.0 * torch.rand(*weight_shape) - 1.0
    j_weight = 2.0 * torch.rand(*weight_shape) - 1.0
    k_weight = 2.0 * torch.rand(*weight_shape) - 1.0

    sum_imaginary_parts = i_weight.abs() + j_weight.abs() + k_weight.abs()

    i_weight = torch.div(i_weight, sum_imaginary_parts)
    j_weight = torch.div(j_weight, sum_imaginary_parts)
    k_weight = torch.div(k_weight, sum_imaginary_parts)

    phase = torch.rand(*weight_shape) * (2 * torch.tensor([np.pi])) - torch.tensor([np.pi])

    r_weight = modulus * np.cos(phase)
    i_weight = modulus * i_weight * np.sin(phase)
    j_weight = modulus * j_weight * np.sin(phase)
    k_weight = modulus * k_weight * np.sin(phase)

    return nn.Parameter(r_weight), nn.Parameter(i_weight), nn.Parameter(j_weight), nn.Parameter(k_weight)

  @staticmethod
  def get_weight_shape(in_channels, out_channels, kernel_size):
    """Construct weight shape based on the input/output channels and kernel size."""
    return (out_channels, in_channels) + kernel_size

  @staticmethod
  def get_kernel_shape(kernel_size, dimension):
    """Construct the kernel shape based on the given dimension and kernel size."""
    if dimension not in QuaternionConvolution.ALLOWED_DIMENSIONS:
      raise InvalidKernelShape('Given dimensions are not allowed.')
    
    if isinstance(kernel_size, int):
      return (kernel_size, ) * dimension

    if isinstance(kernel_size, tuple):
      if len(kernel_size) != dimension:
        raise InvalidKernelShape('Given kernel shape does not match dimension.')

      return kernel_size

    raise InvalidKernelShape('No valid type of kernel size to construct kernel.')

  def __repr__(self):
      return self.__class__.__name__ + '(' \
          + 'in_channels='      + str(self.in_channels) \
          + ', out_channels='   + str(self.out_channels) \
          + ', kernel_size='    + str(self.kernel_size) \
          + ', stride='         + str(self.stride) + ')'


In [ ]:
%%time

class CustomQCNN(nn.Module):
  """Reproduction QCNN to validate quaternion convolution layer."""

  def __init__(self, in_channels, hidden_channels, out_features, kernel_size):
    super(CustomQCNN, self).__init__()

    self.conv_1 = QuaternionConvolution(in_channels, hidden_channels[0], kernel_size, 1)
    self.conv_2 = QuaternionConvolution(hidden_channels[0], hidden_channels[1], kernel_size, 1)

    self.pool_1 = nn.MaxPool2d(2, 2)
    self.dropout_1 = nn.Dropout(0.25)

    self.conv_3 = QuaternionConvolution(hidden_channels[1], hidden_channels[2], kernel_size, 1)
    self.conv_4 = QuaternionConvolution(hidden_channels[2], hidden_channels[3], kernel_size, 1)

    self.pool_2 = nn.MaxPool2d(2, 2)
    self.dropout_2 = nn.Dropout(0.25)

    self.fc_1 = QuaternionLinear(12800, 512)
    self.fc_2 = nn.Linear(512, out_features)

    self.dropout_3 = nn.Dropout(0.5)
    self.sm = nn.Softmax(dim=1)

  def forward(self, x):
    x = F.relu(self.conv_1(x))
    x = F.relu(self.conv_2(x))
    x = self.pool_1(x)
    x = self.dropout_1(x)

    x = F.relu(self.conv_3(x))
    x = F.relu(self.conv_4(x))
    x = self.pool_2(x)
    x = self.dropout_2(x)

    x = torch.flatten(x, start_dim=1) 

    x = F.relu(self.fc_1(x))
    x = self.dropout_3(x)
    x = self.fc_2(x)
    x = self.sm(x)

    return x

# Model parameters
in_channels = 4
hidden_channels = [64, 128, 256, 512]
out_features = 10
kernel_size = (3, 3)

batch_size = 32

custom_qcnn = CustomQCNN(in_channels, hidden_channels, out_features, kernel_size)
custom_qcnn = custom_qcnn.cuda()

print("Number of trainable parameters: ", sum(p.numel() for p in custom_qcnn.parameters() if p.requires_grad))
summary(custom_qcnn, input_size=(in_channels, 32, 32), batch_size=batch_size, device=device.type)

Number of trainable parameters:  2032650
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
QuaternionConvolution-1           [32, 64, 30, 30]              64
QuaternionConvolution-2          [32, 128, 28, 28]             128
         MaxPool2d-3          [32, 128, 14, 14]               0
           Dropout-4          [32, 128, 14, 14]               0
QuaternionConvolution-5          [32, 256, 12, 12]             256
QuaternionConvolution-6          [32, 512, 10, 10]             512
         MaxPool2d-7            [32, 512, 5, 5]               0
           Dropout-8            [32, 512, 5, 5]               0
  QuaternionLinear-9                  [32, 512]             512
          Dropout-10                  [32, 512]               0
           Linear-11                   [32, 10]           5,130
          Softmax-12                   [32, 10]               0
Total params: 6,602
Trainable params: 5,130
Non-tr

In [ ]:
%%time
paper_qcnn_layer = QuaternionConv(4, 64, kernel_size, stride=1)

print("Number of trainable parameters: ", sum(p.numel() for p in paper_qcnn_layer.parameters() if p.requires_grad))
print(paper_qcnn_layer.i_weight[0])

Number of trainable parameters:  640
tensor([[[ 4.9678e-02,  1.7548e-02, -1.5976e-01],
         [-1.5517e-02, -3.2999e-02, -3.3856e-05],
         [ 3.9830e-02, -1.6522e-02, -8.2282e-03]]], grad_fn=<SelectBackward0>)
CPU times: user 7.28 ms, sys: 0 ns, total: 7.28 ms
Wall time: 7.43 ms


In [ ]:
%%time
custom_qcnn_layer = QuaternionConvolution(4, 64, kernel_size, 1)

print("Number of trainable parameters: ", sum(p.numel() for p in custom_qcnn_layer.parameters() if p.requires_grad))
print(custom_qcnn_layer.i_weight[0])

Number of trainable parameters:  640
tensor([[[ 8.9675e-03, -1.2020e-02,  5.8783e-02],
         [-2.6882e-05,  5.7910e-02, -4.7084e-02],
         [ 1.3167e-02,  7.9821e-02,  7.1172e-02]]], grad_fn=<SelectBackward0>)
CPU times: user 4.68 ms, sys: 0 ns, total: 4.68 ms
Wall time: 5.78 ms


In [ ]:
%%time
num_epochs = 80
amount_of_trainings = 3

learning_rate = 0.0001
learning_rate_decay = 1e-6

batch_size = 32

custom_qcnn_accs = []
trainings_seed_excution_time = []

for training_seed in range(amount_of_trainings):
  print(f'Start training seed {training_seed + 1}')
  start_time_training_seed = time.time()

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

  custom_qcnn = CustomQCNN(in_channels, hidden_channels, out_features, kernel_size)
  custom_qcnn = custom_qcnn.cuda()

  optimizer = torch.optim.RMSprop(custom_qcnn.parameters(),lr=learning_rate, weight_decay=learning_rate_decay)
  criterion = nn.CrossEntropyLoss()

  for epoch in range(1, num_epochs):
    
    custom_qcnn.train()

    for index, (x_batch, y_batch) in enumerate(train_loader):
      zeros_channel = torch.zeros((x_batch.shape[0], 1, x_batch.shape[2], x_batch.shape[3]))
      x_batch = torch.cat([x_batch, zeros_channel], dim=1)

      # Check if the input size is correct
      check_input(x_batch)

      x_batch = x_batch.cuda()
      y_batch = y_batch.cuda()
      
      # Perform forward pass
      y_pred = custom_qcnn(x_batch)

      # Compute the loss
      loss = criterion(y_pred, y_batch)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if (epoch / 10).is_integer():
      print (f'Epoch [{epoch + 1}/{num_epochs}], Last loss: {loss.item():.4f}')

  with torch.no_grad():
      n_correct = 0
      n_samples = 0

      custom_qcnn.eval()

      for index, (x_batch, y_batch) in enumerate(test_loader):
        zeros_channel = torch.zeros((x_batch.shape[0], 1, x_batch.shape[2], x_batch.shape[3]))
        x_batch = torch.cat([x_batch, zeros_channel], dim=1)

        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()

        # Check if the input size is correct
        check_input(x_batch)

        # Perform forward pass
        y_pred = custom_qcnn(x_batch)

        _, predicted = torch.max(y_pred,1)
        n_samples += y_batch.size(0)
        n_correct += (predicted == y_batch).sum().item()

      acc = 100 * n_correct / n_samples
      custom_qcnn_accs.append(acc)
  
  elapsed_training_time = int(time.time() - start_time_training_seed)
  trainings_seed_excution_time.append(start_time_training_seed)

  print(f'Finished training seed {training_seed + 1}, accuracy of the network: {acc}%, elapsed time: {elapsed_training_time} sec')

print(f'Average accuracy over {amount_of_trainings}, {num_epochs} epochs each results in: {sum(custom_qcnn_accs) / amount_of_trainings}')

Start training seed 1
Epoch [11/80], Last loss: 2.0787
Epoch [21/80], Last loss: 1.7392
Epoch [31/80], Last loss: 1.6275
Epoch [41/80], Last loss: 1.6913
Epoch [51/80], Last loss: 1.5220
Epoch [61/80], Last loss: 1.6078
Epoch [71/80], Last loss: 1.6412
Finished training seed 1, accuracy of the network: 78.45%, elapsed time: 1957 sec
Start training seed 2
Epoch [11/80], Last loss: 1.9188
Epoch [21/80], Last loss: 1.6543
Epoch [31/80], Last loss: 1.6585
Epoch [41/80], Last loss: 1.5151
Epoch [51/80], Last loss: 1.6406
Epoch [61/80], Last loss: 1.7523
Epoch [71/80], Last loss: 1.4807
Finished training seed 2, accuracy of the network: 77.72%, elapsed time: 1963 sec
Start training seed 3
Epoch [11/80], Last loss: 1.9496
Epoch [21/80], Last loss: 1.7193
Epoch [31/80], Last loss: 1.6582
Epoch [41/80], Last loss: 1.6938
Epoch [51/80], Last loss: 1.5250
Epoch [61/80], Last loss: 1.7079
Epoch [71/80], Last loss: 1.5793
Finished training seed 3, accuracy of the network: 78.14%, elapsed time: 1954